In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import SimplePreProcessor
from loader_util.preprocessing import PatchPreprocessor
from loader_util.preprocessing import MeanPreprocessor
from loader_util.io import HDF5DatasetGenerator
from loader_util.callbacks import TrainingMonitor
from loader_util.nn.conv import AlexNet
##
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

In [ ]:
# construct the trian image generator
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
# load the RGB means
DATASET_MEAN_PATH = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap10/dogs_cats_mean.json"
means = json.loads(open(DATASET_MEAN_PATH).read())

# init the image processors
sp = SimplePreProcessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [ ]:
# init the train and validation dataset generators
TRAIN_HDF5_PATH = r"/content/drive/MyDrive/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/train.hdf5"
VAL_HDF5_PATH = r"/content/drive/MyDrive/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/val.hdf5"
TEST_HDF5_PATH = r"/content/drive/MyDrive/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/test.hdf5"
train_gen = HDF5DatasetGenerator(db_path=TRAIN_HDF5_PATH,
                                 batch_size=128,
                                 aug=aug,
                                 preprocessors=[pp, mp, iap],
                                 classes=2)

val_gen = HDF5DatasetGenerator(db_path=VAL_HDF5_PATH,
                               batch_size=128,
                               preprocessors=[sp, mp, iap],
                               classes=2)

In [ ]:
print(f"[INFO] compiling model......")
OUTPUT_DIR = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap10/"
opt = Adam(learning_rate=0.001)
model = AlexNet.build(width=227, height=227, depth=3,
                      classes=2, reg=0.0002)
model.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

# construct the path of call backs
path = os.path.sep.join([OUTPUT_DIR, f"{os.getpid()}.png"])
callbacks = [TrainingMonitor(path)]


[INFO] compiling model......


In [ ]:
# train the network
H = model.fit_generator(train_gen.generator(),
                        steps_per_epoch=train_gen.num_images // 128,
                        validation_data=val_gen.generator(),
                        validation_steps=val_gen.num_images // 128,
                        epochs=75,
                        max_queue_size=10,
                        callbacks=callbacks,
                        verbose=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/75
156/156 [==============================] - 457s 3s/step - loss: 3.2617 - accuracy: 0.5848 - val_loss: 2.7927 - val_accuracy: 0.6003
Epoch 2/75
156/156 [==============================] - 340s 2s/step - loss: 2.3937 - accuracy: 0.6553 - val_loss: 2.7420 - val_accuracy: 0.5111
Epoch 3/75
156/156 [==============================] - 342s 2s/step - loss: 1.8148 - accuracy: 0.6849 - val_loss: 1.6737 - val_accuracy: 0.6797
Epoch 4/75
156/156 [==============================] - 343s 2s/step - loss: 1.3980 - accuracy: 0.7228 - val_loss: 1.2595 - val_accuracy: 0.7533
Epoch 5/75
156/156 [==============================] - 347s 2s/step - loss: 1.0931 - accuracy: 0.7572 - val_loss: 0.9421 - val_accuracy: 0.7952
Epoch 6/75
156/156 [==============================] - 344s 2s/step - loss: 0.8967 - accuracy: 0.7874 - val_loss: 0.7916 - val_accuracy: 0.8232
Epoch 7/75
156/156 [==============================] - 346s 2s/step - loss: 0.7862 - accuracy: 0.8049 - val_loss: 0.7371 - val_accuracy: 0.8265

In [ ]:
print(f"[INFO] serializing model......")
model.save(OUTPUT_DIR, overwrite=True)

# close the datasets
train_gen.close()
val_gen.close()

[INFO] serializing model......
